In [2]:
import pandas as pd
import requests
import re
pd.set_option('display.max_columns', None)
import os


In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
sp_clave = os.getenv("client_id")
sp_contraseña = os.getenv("client_secret")

In [6]:
pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=sp_clave,
                                               client_secret=sp_contraseña,
                                               redirect_uri="https://www.google.com",
                                               scope="user-library-read"))


In [9]:
canciones_rock = {"nombre_artista": [], "nombre_cancion": [], "año_lanzamiento": [], "género_musical": [], "tipo": []}
for anio in range(2019,2024):
    for offset in range(0, 250, 50):
        datos_rock = sp.search(q=f"genre:rock, year:{anio}", type="track", limit=50, offset=offset)        
        for cancion in datos_rock["tracks"]["items"]:
            canciones_rock["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
            canciones_rock["nombre_cancion"].append(cancion["name"])
            canciones_rock["género_musical"].append("rock")
            canciones_rock["año_lanzamiento"].append(cancion["album"]["release_date"][0:4])
            canciones_rock["tipo"].append(cancion["type"])
df_rock = pd.DataFrame(canciones_rock)
df_rock

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,MAN WITH A MISSION,Dark Crow,2019,rock,track
1,Bring Me The Horizon,MANTRA,2019,rock,track
2,Bring Me The Horizon,in the dark,2019,rock,track
3,ONE OK ROCK,Stand Out Fit In,2019,rock,track
4,YUNGBLUD,parents,2019,rock,track
...,...,...,...,...,...
1245,Cooper Alan,To the Bar,2023,rock,track
1246,Evening Elephants,Snow on The Bluff,2023,rock,track
1247,jaydes,rose,2023,rock,track
1248,Marlon Funaki,Metaphor,2023,rock,track


In [10]:
df_rock.to_csv("rockdatabase.csv")

In [11]:
df_values=pd.read_csv("rockdatabase.csv",index_col=0)
# ponemos el index_col= 0 para q no nos cree el Unnamed

In [12]:
df_values
# el índice va a salir pero lo quitamos al hacer el itertuples

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,MAN WITH A MISSION,Dark Crow,2019,rock,track
1,Bring Me The Horizon,MANTRA,2019,rock,track
2,Bring Me The Horizon,in the dark,2019,rock,track
3,ONE OK ROCK,Stand Out Fit In,2019,rock,track
4,YUNGBLUD,parents,2019,rock,track
...,...,...,...,...,...
1245,Cooper Alan,To the Bar,2023,rock,track
1246,Evening Elephants,Snow on The Bluff,2023,rock,track
1247,jaydes,rose,2023,rock,track
1248,Marlon Funaki,Metaphor,2023,rock,track


In [13]:
values = list(df_values.itertuples(index=False, name=None))

In [14]:
import mysql.connector
from mysql.connector import errorcode

In [15]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "INSERT INTO Spotify (nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s,%s)"
val = values

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

ProgrammingError: 1049 (42000): Unknown database 'musictrends'